# Exploratory Data Analysis

Do a very brief exploratory data analysis (EDA) to understand the data.
- Which file to use?
- Which columns to use as features and which to use as target?
- Are there any missing values or other issues?
- What kind of model can be trained?

Keep in mind that this project doesn't need sophisticated model.
It is about MLOps, not machine learning engineering.
I will try to train a very simple model as fast as possible, then go on with the
actual objective.
So, there also won't be any fancy feature engineering right here.
Just basic data cleaning followed by feature selection and training a basic
model.

## Available Data Files

- hltb_dataset.csv – full unfiltered dataset with all collected entries (games, DLCs, multiplayer titles).
- hltb_dataset_filtered.csv – only entries with at least one recorded playtime.
- hltb_game.csv – standalone full games with reported playtimes.
- hltb_dlc_expansion.csv – downloadable content and expansions with reported playtimes.
- hltb_multiplayer_focused.csv – multiplayer-focused titles with reported playtimes.

Based on the description of the data files, `hltb_game.csv` is likely going to
be the most relevant file.
I am not interested in DLCs or multiplayer titles specifically.


In [44]:
# dependencies
import numpy as np
import pandas as pd

import mlflow
import mlflow.sklearn
from mlflow import MlflowClient

from pathlib import Path
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

In [45]:
# construct paths
PATH_REPO = Path("..").resolve()
PATH_DATA = PATH_REPO / "data" / "raw"
PATH_GAMES = PATH_DATA / "hltb_game.csv"

In [46]:
# set MLflow tracking URI
mlflow.set_tracking_uri("http://localhost:5001")

# set experiment - the artifacts will now use the HTTP API
mlflow.set_experiment("playtime-prediction")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1754334026542, experiment_id='3', last_update_time=1754334026542, lifecycle_stage='active', name='playtime-prediction', tags={}>

In [47]:
# load the data and have a first look at it
data_games = pd.read_csv(PATH_GAMES)
print(data_games.shape)
print(data_games.columns)
data_games.head()

(48787, 22)
Index(['id', 'name', 'type', 'release_date', 'release_precision',
       'release_year', 'release_month', 'release_day', 'main_story_polled',
       'main_story', 'main_plus_sides_polled', 'main_plus_sides',
       'completionist_polled', 'completionist', 'all_styles_polled',
       'all_styles', 'single_player_polled', 'single_player', 'co_op_polled',
       'co_op', 'versus_polled', 'versus'],
      dtype='object')


,id,name,type,release_date,release_precision,release_year,release_month,release_day,main_story_polled,main_story,...,completionist_polled,completionist,all_styles_polled,all_styles,single_player_polled,single_player,co_op_polled,co_op,versus_polled,versus
0,1,688(I) Hunter/Killer,game,1997-07-04,day,1997.0,7.0,4.0,7.0,10.62,...,2.0,15.83,14.0,20.20,7.0,10.62,NaN,NaN,NaN,NaN
1,18,'Splosion Man,game,2009-07-22,day,2009.0,7.0,22.0,32.0,7.60,...,7.0,18.77,60.0,9.47,32.0,7.60,NaN,NaN,NaN,NaN
2,19,.hack//G.U. Vol. 1: Rebirth,game,2006-10-24,day,2006.0,10.0,24.0,75.0,18.95,...,25.0,37.40,178.0,23.97,75.0,18.95,NaN,NaN,NaN,NaN
3,20,.hack//G.U. Vol. 2: Reminisce,game,2007-05-08,day,2007.0,5.0,8.0,41.0,19.65,...,16.0,48.12,88.0,29.08,41.0,19.65,NaN,NaN,NaN,NaN
4,21,.hack//G.U. Vol. 3: Redemption,game,2008-03-25,day,2008.0,3.0,25.0,34.0,21.25,...,8.0,46.08,69.0,27.88,34.0,21.25,NaN,NaN,NaN,NaN


There are 48787 rows in the dataset.
Wow I'm actually surprised that there are this many games lol.
It would be absolutely impossible to play all of them.

Briefly check if there are any duplicates without doing an all-encompassing
check.
ID will be unique, so it does not make sense to check it for duplicates or to
check the whole dataset.
Instead, I will focus on the name of the game.

In [48]:
# check if there are duplicated rows
data_games.duplicated(subset=["name"]).sum()

15

There are 15 games with non-unique names in the dataset.
Briefly inspect them.

In [49]:
# get all rows that have duplicate names (including first occurrences)
duplicate_names = data_games[data_games.duplicated(subset=["name"], keep=False)].sort_values(by="name")
duplicate_names

,id,name,type,release_date,release_precision,release_year,release_month,release_day,main_story_polled,main_story,...,completionist_polled,completionist,all_styles_polled,all_styles,single_player_polled,single_player,co_op_polled,co_op,versus_polled,versus
37538,101945,Aventura Copilului Albastru și Urât,game,2021-02-08,day,2021.0,2.0,8.0,NaN,NaN,...,2.0,1.53,2.0,1.53,NaN,NaN,NaN,NaN,NaN,NaN
34130,88224,Aventura Copilului Albastru și Urât,game,2021-02-08,day,2021.0,2.0,8.0,NaN,NaN,...,2.0,1.33,2.0,1.33,NaN,NaN,NaN,NaN,NaN,NaN
47010,157085,Beyond Shadowgate,game,2024-09-19,day,2024.0,9.0,19.0,4.0,7.98,...,1.0,9.48,7.0,8.18,4.0,7.98,NaN,NaN,NaN,NaN
13941,22324,Beyond Shadowgate,game,1993,year,1993.0,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,5.50,NaN,NaN,NaN,NaN,NaN,NaN
2527,3265,Exile(1988),game,1988-12,month,1988.0,12.0,NaN,6.0,3.62,...,NaN,NaN,7.0,3.77,6.0,3.62,NaN,NaN,NaN,NaN
12474,19146,Exile(1988),game,1988-12,month,1988.0,12.0,NaN,1.0,4.63,...,NaN,NaN,1.0,4.63,1.0,4.63,NaN,NaN,NaN,NaN
3504,4500,Hook(1992),game,1992-07-17,day,1992.0,7.0,17.0,31.0,1.45,...,4.0,1.43,36.0,1.48,31.0,1.45,NaN,NaN,NaN,NaN
9977,13749,Hook(1992),game,1992,year,1992.0,NaN,NaN,7.0,3.45,...,NaN,NaN,8.0,3.83,7.0,3.45,NaN,NaN,NaN,NaN
33857,87116,Long Time No See(2023),game,2023-04-22,day,2023.0,4.0,22.0,1.0,0.50,...,1.0,0.67,3.0,0.58,1.0,0.50,NaN,NaN,NaN,NaN
44266,139677,Long Time No See(2023),game,2023-10-27,day,2023.0,10.0,27.0,3.0,0.83,...,2.0,0.58,6.0,0.72,3.0,0.83,NaN,NaN,NaN,NaN


In [50]:
# check if the ID is the only entry that differs between the duplicates

# remove the ID column
duplicate_names_noID = duplicate_names.drop(columns=["id"])

# now check if all other entries are identical for the duplicates
duplicate_names_noID.duplicated().sum()

0

No they differ in at least one other column, too.
Since this is supposed to be a quick EDA, I will just remove all duplicates and
only keep the first occurrence of each name.

In [51]:
# drop duplicates, keeping the first occurrence of each name
data_games = data_games.drop_duplicates(subset=["name"], keep='first')

# verify duplicates are gone
data_games.duplicated(subset=['name']).sum()

0

No duplicates anymore.


Looking at the available columns I really have to say that this data set is not
really that nice for predictive modeling, but that's fine now.
The model doesn't have to be good.

Something that I could predict is `completionist`.
That's the average playtime for full completion in hours.
This could be based on time of release and playtime to complete main story
and side quests.

There could be some trends in there.
Like if a game has a long story line, but basically no side quests, maybe full
completion will then also be quick and vice versa.

I cannot really use time of release as feature here, because I want to schedule
this or rather filter based on time.
I think if this was just about getting a good model, using time of release could
be one of the best features for this.
While I don't know, I really believe that time of release could contain a lot of
information about the game.
But I just need any model, not a good one, and I also want to be able to
simulate scheduling on new data, so unfortunately I cannot use time of release
as feature.

I will need the following columns:
- id: uniquely identify game, primary key; remove name
- release_year and release month: for scheduling or rather running for a
particular month
- main_story: average time to finish the main story (hours)
- main_story_polled: the number of players submitting the information
- main_plus_sides_polled and main_plus_sides: same for side quests
- completionist: playtime for 100% completion (hours) -> use this as target

Columns not needed:
- name: not necessary for the predictions
- type: already filtered for games only
- release precision: when there is no month, I will remove the entry
- all_styles_polled and all_styles: average playtime across all playstyles and
number of players submitting the information -> contains information from target
- single player, coop and versus columns -> let's just keep it simple here
- completionist_polled: I aim to predict the completionist time. This column
may help to predict the completionist time, but it may leak data, too. Like if
every player who completed the main story also completed it 100%, this number
would be large. If this number is tiny in comparison to the number of players
who completed the main story, this could be a sign that the completionist time
is just extremely long, indicated by people not completing the game.

In [52]:
# select columns I want to use
data_games = data_games[[
    "id",
    "release_year",
    "release_month",
    "main_story",
    "main_story_polled",
    "main_plus_sides",
    "main_plus_sides_polled",
    "completionist",
]]

data_games.head()

,id,release_year,release_month,main_story,main_story_polled,main_plus_sides,main_plus_sides_polled,completionist
0,1,1997.0,7.0,10.62,7.0,35.37,5.0,15.83
1,18,2009.0,7.0,7.60,32.0,9.23,21.0,18.77
2,19,2006.0,10.0,18.95,75.0,24.50,78.0,37.40
3,20,2007.0,5.0,19.65,41.0,31.73,31.0,48.12
4,21,2008.0,3.0,21.25,34.0,30.85,27.0,46.08


Now briefly clean the data.

In [53]:
# get fraction of missing values
data_games.isna().mean()

id                        0.000000
release_year              0.031555
release_month             0.104671
main_story                0.185065
main_story_polled         0.185147
main_plus_sides           0.454605
main_plus_sides_polled    0.454564
completionist             0.361683
dtype: float64

Ugh that's annoying! This is quite a lot.
I assume there would be some ways to impute this, but I want to split the data
based on realease time.
Imputing with overall mean could mess up data from particular years.
I will just drop all rows with missing values, even though that will be a
massive loss of data.

In [54]:
# print number of rows before dropna
print(f"Number of rows before dropna: {data_games.shape[0]}")

# drop rows with missing values
data_games.dropna(inplace=True)

# print number of rows after dropna
print(f"Number of rows after dropna: {data_games.shape[0]}")

Number of rows before dropna: 48772
Number of rows after dropna: 17949


Ouch! That's a lot of data lost.

Anyway let's go on!
I now need to split the data into features and target.
I won't split into a real train test split, so I can predict particular years or
whatever.

In [55]:
data_games

,id,release_year,release_month,main_story,main_story_polled,main_plus_sides,main_plus_sides_polled,completionist
0,1,1997.0,7.0,10.62,7.0,35.37,5.0,15.83
1,18,2009.0,7.0,7.60,32.0,9.23,21.0,18.77
2,19,2006.0,10.0,18.95,75.0,24.50,78.0,37.40
3,20,2007.0,5.0,19.65,41.0,31.73,31.0,48.12
4,21,2008.0,3.0,21.25,34.0,30.85,27.0,46.08
...,...,...,...,...,...,...,...,...
48737,170797,2024.0,5.0,0.73,6.0,0.17,1.0,0.23
48738,170826,2025.0,7.0,1.23,2.0,1.42,3.0,1.33
48750,170972,2025.0,7.0,4.82,10.0,5.43,4.0,7.02
48755,171355,2025.0,7.0,1.55,6.0,1.47,1.0,3.30


Briefly check which years even still have data to see what I can use for
training and prediction.

In [56]:
# check which years are left
np.sort(data_games["release_year"].unique())

array([1958., 1972., 1976., 1977., 1978., 1979., 1980., 1981., 1982.,
       1983., 1984., 1985., 1986., 1987., 1988., 1989., 1990., 1991.,
       1992., 1993., 1994., 1995., 1996., 1997., 1998., 1999., 2000.,
       2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009.,
       2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017., 2018.,
       2019., 2020., 2021., 2022., 2023., 2024., 2025., 2026.])

Ok I will use everything up until and including 2020 as training.
The years after that can be used for prediction.

In [57]:
# make train set

# features
# time of release is not needed anymore after filtering
X_train = data_games[data_games["release_year"] <= 2020][[
    "main_story",
    "main_story_polled",
    "main_plus_sides",
    "main_plus_sides_polled",
]]

# labels
y_train = data_games[data_games["release_year"] <= 2020]["completionist"]

In [58]:
# make test set

# features
# don't drop the time of release here to simulate scheduling on new data
X_test = data_games[data_games["release_year"] > 2020][[
    "release_year",
    "release_month",
    "main_story",
    "main_story_polled",
    "main_plus_sides",
    "main_plus_sides_polled",
]]

# labels
y_test = data_games[data_games["release_year"] > 2020]["completionist"]

Okay now I need to train a model.
I really like random forests, so I will just use that.
It can learn non-linear relationships and is not prone to overfitting, so let's
go.

In [59]:
# start an MLFlow run
run = mlflow.start_run()

model_params = {
    # number of trees: don't go wild here, keep it simple
    # the model doesn't really have to perform
    # I think 100 is a good balance between speed and performance
    "n_estimators": 100,
    # keep forest in check to avoid overfitting
    # maximum depth of trees
    "max_depth": 10,
    # minimum samples to split a node
    "min_samples_split": 5,
    # minimum samples in a leaf
    "min_samples_leaf": 2,
    # features to consider for best split
    "max_features": 'sqrt',
    # for reproducibility
    "random_state": 42,
    # use all available cores
    "n_jobs": -1,
}

# log model parameters
mlflow.log_params(model_params)
mlflow.log_param("model_type", "RandomForestRegressor")

# train a random forest regressor
rf_regressor = RandomForestRegressor(**model_params)

# train the model
rf_regressor.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,5
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


Ok soooo this trained like 0.1 seconds, which is amazing!

Now I need a function that filters the data based on release time to get some
data to test this on.

In [60]:
def filter_by_release(
    year: int,
    month: int | None = None,
    data: pd.DataFrame = data_games,
) -> tuple[pd.DataFrame, pd.Series]:
    """
    Filter the data based on release time to get data to run the model on.
    
    Args:
        year (int): The year to filter by.
        month (int | None): The month to filter by. If None, the whole year is used.
        data (pd.DataFrame): The data to filter.

    Returns:
        X_test (pd.DataFrame): The test features.
        y_test (pd.Series): The test labels.
    """
    # filter by year
    data = data[data["release_year"] == year]

    # optionally filter by month
    if month is not None:
        data = data[data["release_month"] == month]

    # get test features
    X_test = data[[
        "main_story",
        "main_story_polled",
        "main_plus_sides",
        "main_plus_sides_polled",
    ]]

    # get test labels
    y_test = data["completionist"]

    return X_test, y_test

Cool, let's check it out!

In [61]:
# test the function on the next year
# was trained until 2020, so test for 2021
X_test_2021, y_test_2021 = filter_by_release(year=2021)

# check the first few rows
X_test_2021.head()

,main_story,main_story_polled,main_plus_sides,main_plus_sides_polled
6975,125.00,2.0,300.00,2.0
12074,0.87,20.0,2.30,4.0
13279,0.97,14.0,2.00,1.0
14168,4.75,10.0,6.13,3.0
15025,1.45,32.0,2.75,2.0


Great! This seems to work.

The I guess we can make some predictions now.

Make another function wrapping around filtering the data,making predictions
and evaluating the model.

In [62]:
def make_predictions(
    year: int,
    month: int | None = None,
) -> float:
    """
    Make predictions for a given month.
    """
    # filter data
    
    if month is None:
        X_test, y_test = filter_by_release(year=year)
    else:
        X_test, y_test = filter_by_release(year=year, month=month)

    # make predictions
    y_pred = rf_regressor.predict(X_test)

    # evaluate
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    
    return rmse, y_pred

In [63]:
# test the model
rmse_2021, y_pred = make_predictions(year=2021)

# print result
print(f"RMSE for 2021: {rmse_2021}")

# log the result
mlflow.log_metric("rmse_2021", rmse_2021)

# log the model
mlflow.sklearn.log_model(
    rf_regressor,
    "random_forest_regressor"
)

# end the MLFlow run
mlflow.end_run()

RMSE for 2021: 68.92833952755117


In [65]:
print(f"Mean playtime for 2021: {y_test_2021.mean()}")
print(f"Median playtime for 2021: {y_test_2021.median()}")
print(f"Standard deviation of playtime for 2021: {y_test_2021.std()}")

Mean playtime for 2021: 29.48776176753122
Median playtime for 2021: 9.2
Standard deviation of playtime for 2021: 93.97233880491784


Phew, this is performance is really not what you'd typically
want to see.

The RMSE is about 2.3 times the mean and about 7.5 times the median.
This is bad.
It is just 73% of the standard deviation though, which is kind
of acceptable.

So this mean the model really doesn't capture the bulk of the
data.
Stuff like log transformations may have helped, but who knows.

As I mentioned right away at the beginning, the model was never meant to be
good.
It was just meant to be something that can make predictions on new data and can
be used in MLOps pipelines.
This training was done with the intent of just getting any model very fast on
data that has some time of release information.
Unfortunately, this also meant that this time information could not be used as
features, which likely made the model perform even worse.

So, this is actually perfect!
It fulfills its purpose.

Now I will go on and refactor this code into scripts and continue with the
actual MLOps part.